In [ ]:
# @title Imports

import os
import math
import time
import torch
import nbimporter
import matplotlib.pyplot as plt

from transformers import AutoTokenizer
from tempfile import TemporaryDirectory
from models._utils import NeuralTransformer
from datasets import load_dataset as load_hf_dataset
from utils import DEVICE, BLOCK_SIZE, NUM_TOKENS, init_random_seeds
from CreateSyntheticDataset import tokenize_and_chunk  # works because of nbimporter

## Language modeling with tiny Shakespeare

In [ ]:
# @title HuggingFace Tokenizers
# @markdown Note there are two ways to call the tokenizer's encoder.

tokenizer = AutoTokenizer.from_pretrained("google/byt5-large")

expl_text = "Welcome to the 🤗 Tokenizers library."
impl_text = "We are very happy to show you the 🤗 Transformers library."
expl_encode = tokenizer.encode(expl_text)
impl_encode = tokenizer(impl_text)
print(
    f"Calling `tokenizer.encode(text)`:\n\ttext: {expl_text}\n\ttokenized: {expl_encode}\n\tdecoded: {tokenizer.decode(expl_encode)}",
    end="\n\n",
)
print(
    f"Calling `tokenizer(text)`:\n\tobject.keys(): {impl_encode.keys()}\n\ttext: {impl_text}\n\ttokenized: {impl_encode['input_ids']}\n\tdecoded: {tokenizer.decode(impl_encode['input_ids'])}",
    end="\n\n",
)

In [ ]:
# @title HuggingFace Datasets

text_dataset = load_hf_dataset("tiny_shakespeare")
print(text_dataset, end="\n\n")
print("~~~" * 50, end="\n\n")

print(
    "train:",
    type(text_dataset["train"]["text"]),
    len(text_dataset["train"]["text"]),
    type(text_dataset["train"]["text"][0]),
    len(text_dataset["train"]["text"][0]),
    end="\n\n",
)
print(
    "validation:",
    type(text_dataset["validation"]["text"]),
    len(text_dataset["validation"]["text"]),
    type(text_dataset["validation"]["text"][0]),
    len(text_dataset["validation"]["text"][0]),
    end="\n\n",
)
print(
    "test:",
    type(text_dataset["test"]["text"]),
    len(text_dataset["test"]["text"]),
    type(text_dataset["test"]["text"][0]),
    len(text_dataset["test"]["text"][0]),
    end="\n\n",
)

In [ ]:
# @title Tokenization and Chunking
# @markdown Apply the tokenization and chunking to each split.

text_dataset = text_dataset.map(
    tokenize_and_chunk, batched=True, fn_kwargs=dict(tokenizer=tokenizer)
)
print(text_dataset, end="\n\n")
print("~~~" * 50, end="\n\n")

print(
    "text_dataset['train']['input_ids']:\n",
    "\ttype:",
    type(text_dataset["train"]["input_ids"]),
    "\n\tlength:",
    len(text_dataset["train"]["input_ids"]),
    end="\n\n",
)
print(
    "text_dataset['train']['input_ids'][0]:\n",
    "\ttype:",
    type(text_dataset["train"]["input_ids"][0]),
    "\n\tlength:",
    len(text_dataset["train"]["input_ids"][0]),
    end="\n\n",
)
print(
    "text_dataset['train']['input_ids'][0][0]:\n",
    "\ttype:",
    type(text_dataset["train"]["input_ids"][0][0]),
    "\n\tvalue:",
    text_dataset["train"]["input_ids"][0][0],
    end="\n\n",
)
print("~~~" * 50, end="\n\n")

print(f"Original sequence (text):\n\t{text_dataset['train']['text'][0]}", end="\n\n")
print(
    f"Encoded sequence (tokens):\n\t {text_dataset['train']['input_ids'][0]}",
    end="\n\n",
)
print(
    f"Decoded sequence (tokens):\n\t {tokenizer.decode(text_dataset['train']['input_ids'][0])}",
    end="\n\n",
)

In [ ]:
# @title Define the Transformer model


class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 1000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)  # batch_first=True
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
        """
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)


class TransformerModel(torch.nn.Module):
    def __init__(
        self,
        ntoken: int,
        d_model: int,
        nhead: int,
        d_hid: int,
        nlayers: int,
        dropout: float = 0.5,
    ):
        super().__init__()
        self.model_type = "Transformer"
        self.d_model = d_model
        self.embedding = torch.nn.Embedding(ntoken, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = torch.nn.TransformerEncoderLayer(
            d_model, nhead, d_hid, dropout, batch_first=True
        )
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layers, nlayers)
        self.linear = torch.nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: torch.Tensor, src_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Arguments:
            src: Tensor, shape [batch_size, seq_len]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [batch_size, seq_len, ntoken]
        """
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        if src_mask is None:
            src_mask = torch.nn.Transformer.generate_square_subsequent_mask(
                src.size(1)  # Use src.size(1) to get the seq_len
            ).to(
                src.device
            )  # Use src.device to match device of src
        output = self.transformer_encoder(src, src_mask)
        output = self.linear(output)
        return output

    @torch.no_grad()
    def generate(
        self,
        idx: torch.LongTensor,
        max_new_tokens: int,
        temperature=1.0,
        top_k=None,
    ):
        """
        Special generate method for the Transformer model.
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Since we trained the model to directly predict the next token we take the index as the argmin
        over the distance between the output and the embedding table.
        """
        # Set model to evaluation mode
        self.eval()

        # Loop through time
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= BLOCK_SIZE else idx[:, -BLOCK_SIZE:]
            # forward the model to get the output
            outputs = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = outputs[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float("Inf")
            # apply softmax to convert logits to (normalized) probabilities
            probs = torch.nn.functional.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1).view(1, 1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [ ]:
# @title Create token datasets

train_dataset = [torch.LongTensor(sequence) for sequence in text_dataset["train"]["input_ids"]]
validation_dataset = [
    torch.LongTensor(sequence) for sequence in text_dataset["validation"]["input_ids"]
]
test_dataset = [torch.LongTensor(sequence) for sequence in text_dataset["test"]["input_ids"]]

# get a test sample for an example
data = test_dataset[0].unsqueeze(0)
print("data: ", data.shape, data.dtype, data.requires_grad, data.device, end="\n\n")

In [ ]:
# @title Instantiate a TransformerModel

ntokens = tokenizer.vocab_size
emsize = 302  # embedding dimension
d_hid = 512  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 1  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention (NOTE: nhead must be a divisor of d_hid)
dropout = 0.1  # dropout probability

model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(DEVICE)
print(f"Vocab size (i.e. num tokens) = {ntokens}")
print(f"Number of attn heads = {nhead}\n")

In [ ]:
# @title Train the Transformer model


def train(model: torch.nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.0
    log_interval = 300
    start_time = time.time()
    global epoch, optimizer, scheduler, criterion

    num_batches = len(train_dataset)
    for batch in range(num_batches):
        tokens = train_dataset[batch].unsqueeze(0)
        # parse into input and target
        input = tokens[:, :-1].to(DEVICE)  # ``[batch_size=1, seq_len]``
        target = tokens[:, 1:].reshape(-1).to(DEVICE)  # ``[batch_size=1 * seq_len]``
        # forward pass
        output = model(input)  # ``[batch_size=1, seq_len, ntokens]``
        output_flat = output.view(-1, ntokens)  # ``[batch_size=1 * seq_len, ntokens]``
        # backpropagation step
        optimizer.zero_grad()
        loss = criterion(output_flat, target)
        # check if the computed loss requires gradient
        if loss.requires_grad:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(
                f"| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | "
                f"lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | "
                f"loss {cur_loss:5.2f} | ppl {ppl:8.2f}"
            )
            total_loss = 0
            start_time = time.time()


def evaluate(model: torch.nn.Module) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.0
    with torch.no_grad():
        num_batches = len(validation_dataset)
        for batch in range(num_batches):
            tokens = validation_dataset[batch].unsqueeze(0)
            input = tokens[:, :-1].to(DEVICE)
            target = tokens[:, 1:].reshape(-1).to(DEVICE)
            output = model(input)
            output_flat = output.view(-1, ntokens)
            total_loss += criterion(output_flat, target).item()
    return total_loss / num_batches

In [ ]:
# @markdown Loop over epochs. Save the model if the validation loss is the best we’ve seen so far. Adjust the learning rate after each epoch.

criterion = torch.nn.CrossEntropyLoss()  # loss function
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.99)

best_val_loss = float("inf")
epochs = 0

# Load the previously saved best model checkpoint if it exists
final_model_params_path = os.path.join("../models/", "shakespeare_transformer_model.pt")
if os.path.exists(final_model_params_path):
    print("Loading a previously saved model checkpoint...\n")
    model.load_state_dict(torch.load(final_model_params_path))
    best_val_loss = evaluate(model)
    print(f"Previous validation loss: \t {best_val_loss:5.2f}\n")

# Train the model
if epochs > 0:
    print(f"Training for {epochs} epoch(s)...\n")
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

        for epoch in range(1, epochs + 1):
            epoch_start_time = time.time()
            train(model)
            print(f"TRAIN done.\n")
            val_loss = evaluate(model)
            print(f"EVAL done.\n")
            val_ppl = math.exp(val_loss)
            elapsed = time.time() - epoch_start_time
            print("-" * 89)
            print(
                f"| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | "
                f"valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}"
            )
            print("-" * 89)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print(f"Saving the newest best model...\n")
                torch.save(model.state_dict(), best_model_params_path)

            scheduler.step()

        if os.path.exists(best_model_params_path):
            print(f"Reloading the overall best model...\n")
            model.load_state_dict(torch.load(best_model_params_path))  # reload the final best model
            print(f"Saving the overall best model to permanent location...\n")
            torch.save(
                model.state_dict(),
                final_model_params_path,
            )  # save the final best model

In [ ]:
# @title Generate new text

max_new_tokens = 100
idx = test_dataset[0][:-1].unsqueeze(0).to(DEVICE)
idx_gen = model.generate(idx, max_new_tokens, top_k=None)

print(idx.shape, idx_gen.shape, end="\n\n")
print(tokenizer.decode(idx.tolist()[0]), end="\n\n")
print(tokenizer.decode(idx_gen.tolist()[0][-max_new_tokens:]), end="\n\n")

## Neural data modeling

In [ ]:
# @title Create neural datasets
# @markdown A synthetic dataset where the neural activity is the embeddings of tokens from tiny Shakespeare.

init_random_seeds()  # set random seeds

ntokens = tokenizer.vocab_size
emsize = 302
d_hid = 512
embedding = torch.nn.Embedding(ntokens, emsize, _freeze=True)  # fixed embedding map

# Create datasets
train_dataset = [
    torch.vstack([embedding(token) for token in torch.LongTensor(sequence)])
    for sequence in text_dataset["train"]["input_ids"]
]
validation_dataset = [
    torch.vstack([embedding(token) for token in torch.LongTensor(sequence)])
    for sequence in text_dataset["validation"]["input_ids"]
]
test_dataset = [
    torch.vstack([embedding(token) for token in torch.LongTensor(sequence)])
    for sequence in text_dataset["test"]["input_ids"]
]

# get a test sample for an example
data = test_dataset[0].unsqueeze(0)
print("data: ", data.shape, data.dtype, data.requires_grad, data.device, end="\n\n")

mask = torch.ones(emsize, dtype=torch.bool).unsqueeze(0)
print("mask:", mask.shape, mask.dtype, mask.requires_grad, mask.device, end="\n\n")

In [ ]:
# DEBUG

# What tokens and their corresponding characters are in the training set
real_train_tokens = set()
for sequence in text_dataset["train"]["input_ids"]:
    tokens = set(sequence[:-1])
    real_train_tokens.update(tokens)
print(len(real_train_tokens), "\t", real_train_tokens)
print("\t", tokenizer.decode(list(real_train_tokens)))
print("\n", "~" * 333, "\n")

In [ ]:
# @title Instantiate a NeuralTransformer model

model = NeuralTransformer(
    input_size=emsize,
    hidden_size=d_hid,
    version_2=True,
    num_tokens=ntokens,
    vq_vae=False,
).to(DEVICE)
# NOTE: In reality we don't actually know the underlying vocabulary size (i.e. num_tokens) of the embedding table used to generated the neural data.

print(f"Vocab size (i.e. num tokens) = {ntokens}")
print(f"Model internal tokens = {model.num_tokens}")
print(f"Number of attn heads = {model.num_heads}\n")

In [ ]:
# @title Test NeuralTransformer model's input-output functionality

mask = mask.to(DEVICE)
input = data[:, :-1, :].to(DEVICE)
target = data[:, 1:, :].to(DEVICE)
with torch.no_grad():
    output = model(input, mask)


print(
    "input:",
    input.shape,
    input.dtype,
    input.requires_grad,
    input.device,
    end="\n\n",
)
print(
    "target:",
    target.shape,
    target.dtype,
    target.requires_grad,
    target.device,
    end="\n\n",
)
print(
    "output:",
    output.shape,
    output.dtype,
    output.requires_grad,
    output.device,
    end="\n\n",
)

In [ ]:
# @title Test NeuralTransformer model's internal tokenizer

# The code below should only work when we cheat and set the codebook to be exactky the embedding table that was used to generate the neural data.
# This is because the codebook is initialized randomly. If the model has not been trained then there is no reason for its internal tokenizer to
# correctly invert the ground-truth embedding, which should be unknown to us. Thereforem the goal of our optimization is ultimately to learn the
# a codebook that is as close as possible to the ground-truth but unknown embedding map.

if embedding.weight.shape == model.codebook.shape:
    assert not torch.allclose(
        embedding.weight, model.codebook.cpu()
    ), "The codebook should be different from the embedding map!"

# Replace model codebook with the embedding map use to generate the dataset
tmp = model.codebook  # save for later restoration
model.codebook = torch.nn.Parameter(embedding.weight.to(DEVICE))  # let's cheat

assert torch.allclose(
    embedding.weight, model.codebook.cpu()
), "The codebook should be the same as the embedding map!"

# Get some ground-truth test data
token_list = text_dataset["test"]["input_ids"][0]
token_target = torch.LongTensor(token_list)
neural_target = torch.vstack([embedding(t) for t in token_target])

# Compare the tokenized and retokenized sequences when the codebook is the true embedding map
with torch.no_grad():  # model tokenizer takes in a neural sequence and outputs a token sequence
    retokenized_target = model.tokenize_neural_data(neural_target.unsqueeze(0)).squeeze(0)
print(type(token_list), neural_target.shape, token_target.shape, retokenized_target.shape)
print("\t", token_target[:5], retokenized_target[:5], end="\n\n")

# The tokenized and retokenized sequences should be the same
assert torch.allclose(
    token_target, retokenized_target
), "The tokenized and retokenized sequences should be the same!"

# Restore the model codebook to its original random initialization
model.codebook = tmp

if embedding.weight.shape == model.codebook.shape:
    assert not torch.allclose(embedding.weight, model.codebook.cpu())

# Compare the tokenized and retokenized sequences when the codebook is NOT the true embedding map
with torch.no_grad():  # model tokenizer takes in a neural sequence and outputs a token sequence
    retokenized_target = model.tokenize_neural_data(neural_target.unsqueeze(0)).squeeze(0)
print(type(token_list), neural_target.shape, token_target.shape, retokenized_target.shape)
print("\t", token_target[:5], retokenized_target[:5], end="\n\n")

# The tokenized and retokenized sequences should NOT be the same
assert not torch.allclose(
    token_target, retokenized_target
), "The tokenized and retokenized sequences should NOT be the same!"

In [ ]:
# @title Train the NeuralTransformer model


def train(model: torch.nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.0
    log_interval = 300
    start_time = time.time()
    global epoch, optimizer, scheduler

    num_batches = len(train_dataset)
    for batch in range(num_batches):
        data = train_dataset[batch].unsqueeze(0)
        mask = mask = torch.ones(emsize, dtype=torch.bool).unsqueeze(0)
        # parse into input and target
        mask = mask.to(DEVICE)
        input = data[:, :-1, :].to(DEVICE)
        target = data[:, 1:, :].to(DEVICE)
        # forward pass
        output = model(input, mask)  # ``[batch_size=1, seq_len, ntokens]``
        # backpropagation step
        optimizer.zero_grad()
        loss = model.loss_fn()(
            output, target, mask
        )  # flattens output to ``[batch_size=1 * seq_len, ntokens]``

        # check if the computed loss requires gradient
        if loss.requires_grad:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            try:
                ppl = math.exp(cur_loss)
            except OverflowError:
                ppl = float("inf")
            print(
                f"| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | "
                f"lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | "
                f"loss {cur_loss:5.2f} | ppl {ppl:8.2f}"
            )
            total_loss = 0
            start_time = time.time()


def evaluate(model: torch.nn.Module) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.0
    with torch.no_grad():
        num_batches = len(validation_dataset)
        for batch in range(num_batches):
            data = validation_dataset[batch].unsqueeze(0)
            mask = mask = torch.ones(emsize, dtype=torch.bool).unsqueeze(0)
            mask = mask.to(DEVICE)
            input = data[:, :-1, :].to(DEVICE)
            target = data[:, 1:, :].to(DEVICE)
            output = model(input, mask)
            loss = model.loss_fn()(output, target, mask)
            total_loss += loss.item()

    return total_loss / num_batches

In [ ]:
# DEBUG

print(len(model.token_neural_map.unique(dim=0)) - 1, end="\n\n")
print("codebook trainable?", model.codebook.requires_grad, end="\n\n")
print("codebook trained?", model.codebook.grad is not None, end="\n\n")

In [ ]:
# DEBUG

A = embedding.weight.numpy()
B = model.codebook.detach().cpu().numpy()
C = model.token_neural_map.detach().cpu().numpy()

print(A.shape, B.shape, C.shape)

plt.imshow(A @ A.T)
plt.imshow(A.T @ A)
plt.imshow(A[:5, :10])
plt.title("embedding")
plt.ylabel("token index")
plt.xlabel("input dimension")
plt.show()

plt.imshow(B @ B.T)
plt.imshow(B.T @ B)
plt.imshow(B[:5, :10])
plt.title("codebook")
plt.ylabel("token index")
plt.xlabel("input dimension")
plt.show()

plt.imshow(C @ C.T)
plt.imshow(C.T @ C)
plt.imshow(C[:5, :10])
plt.title("token_neural_map")
plt.ylabel("token index")
plt.xlabel("input dimension")
plt.show()

In [ ]:
# @markdown Loop over epochs. Save the model if the validation loss is the best we’ve seen so far. Adjust the learning rate after each epoch.

lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.99)

best_val_loss = float("inf")
epochs = 20

# Load the previously saved best model checkpoint if it exists
final_model_params_path = os.path.join("../models/", "neural_transformer_model.pt")
if os.path.exists(final_model_params_path):
    print("Loading a previously saved model checkpoint...\n")
    model.load_state_dict(torch.load(final_model_params_path))
    best_val_loss = evaluate(model)
    print(f"Previous validation loss: \t {best_val_loss:5.2f}\n")

# Train the model
if epochs > 0:
    print(f"Training for {epochs} epoch(s)...\n")
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

        for epoch in range(1, epochs + 1):
            epoch_start_time = time.time()
            train(model)
            print(f"TRAIN done.\n")
            val_loss = evaluate(model)
            print(f"EVAL done.\n")
            try:
                val_ppl = math.exp(val_loss)
            except OverflowError:
                val_ppl = float("inf")
            elapsed = time.time() - epoch_start_time
            print("-" * 89)
            print(
                f"| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | "
                f"valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}"
            )
            print("-" * 89)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print(f"Saving the newest best model...\n")
                torch.save(model.state_dict(), best_model_params_path)

            scheduler.step()

        if os.path.exists(best_model_params_path):
            print(f"Reloading the overall best model...\n")
            model.load_state_dict(torch.load(best_model_params_path))  # reload the final best model
            print(f"Saving the overall best model to permanent location...\n")
            torch.save(
                model.state_dict(),
                final_model_params_path,
            )  # save the final best model

In [ ]:
# DEBUG

print(len(model.token_neural_map.unique(dim=0)) - 1, end="\n\n")
print("codebook trainable?", model.codebook.requires_grad, end="\n\n")
print("codebook trained?", model.codebook.grad is not None, end="\n\n")

In [ ]:
# DEBUG

A = embedding.weight.numpy()
B = model.codebook.detach().cpu().numpy()
C = model.token_neural_map.detach().cpu().numpy()

print(A.shape, B.shape, C.shape)

plt.imshow(A.T @ A)  # (input_size, input_size)
plt.imshow(A[:5, :10])
plt.title("embedding")
plt.ylabel("token index")
plt.xlabel("input dimension")
plt.show()

plt.imshow(B.T @ B)  # (input_size, input_size)
plt.imshow(B[:5, :10])
plt.title("codebook")
plt.ylabel("token index")
plt.xlabel("input dimension")
plt.show()

plt.imshow(C.T @ C)  # (input_size, input_size)
plt.imshow(C[:5, :10])
plt.title("token_neural_map")
plt.ylabel("token index")
plt.xlabel("input dimension")
plt.show()

In [ ]:
# @title Generate new data

data = test_dataset[0][:-1, :].unsqueeze(0).to(DEVICE)
mask = torch.ones(emsize, dtype=torch.bool).unsqueeze(0).to(DEVICE)
max_new_tokens = 100
data_gen = model.generate(data, mask, max_new_tokens, autoregressive=True, top_k=None)
print(mask.shape, data.shape, data_gen.shape, embedding.weight.shape, end="\n\n")

In [ ]:
# @markdown We want to tokenize the neural data.
# An oracle told us that the neural data itself is an embedding of tokens from some unknown vocabulary.
# We can do this by using the tokenize_neural_data method of our model.

# First run a test on data for which we know what the true token output should be.
# This is just to confirm if our tokenize_neural_data method is working as expected.
with torch.no_grad():
    sequence = data
    inp_tokens = model.tokenize_neural_data(
        neural_sequence=data,
        feature_mask=mask,
        token_matrix=embedding.weight,
    )
    # iff correct these two should match
    print(text_dataset["test"]["input_ids"][0], end="\n\n")  # ground-truth tokens
    print(text_dataset["test"]["text"][0], end="\n\n")  # ground-truth text
    print(inp_tokens.squeeze().tolist(), end="\n\n")  # decoded tokens
    print(tokenizer.decode(inp_tokens.squeeze().tolist()), end="\n\n")  # decoded text

In [ ]:
# Now do the same thing on the newly generated data
with torch.no_grad():
    sequence = data_gen
    gen_tokens = model.tokenize_neural_data(
        neural_sequence=sequence,
        feature_mask=mask,
        token_matrix=embedding.weight,
    )
    print(gen_tokens.squeeze().tolist(), end="\n\n")  # decoded tokens
    print(tokenizer.decode(gen_tokens.squeeze().tolist()), end="\n\n")  # decoded text

In [ ]:
# DEBUG

# Assert should not be raised if the model learned to correctly map the tokens in the training  set
for idx in range(data_gen.shape[1]):
    neural = data_gen[:, [idx], :]
    print("sequence time index:", idx, "\nneural:", neural.squeeze()[:5], end="\n\n")

    ct = model.tokenize_neural_data(neural).item()
    coded = model.codebook[torch.tensor(ct, dtype=torch.long)].detach()
    mapped = model.token_neural_map[torch.tensor(ct, dtype=torch.long)]
    print("codebook token index:", ct, "\ncoded:", coded[:5], "\nmapped:", mapped[:5], end="\n\n")

    assert torch.allclose(neural, mapped), "Basic check failed; Inconsistency in mapping!"

    et = model.tokenize_neural_data(neural, token_matrix=embedding.weight).item()
    embedded = embedding(torch.tensor(et, dtype=torch.long))
    print(
        "embedding token index:",
        et,
        "\ncharacter:",
        tokenizer.decode([et]),
        "\nin train set:",
        et in real_train_tokens,
        "\nembedded:",
        embedded[:5],
        end="\n\n",
    )

    if et in real_train_tokens:
        ### DEBUG ###
        assert torch.any(neural != 0), "Model did not learn to map a vector it was trained on!"
        assert torch.allclose(
            embedded, mapped.cpu()
        ), "The learned mapping did not converge to the true embedding!"
        ### DEBUG ###

    print("~" * 99, end="\n\n")

In [ ]:
# DEBUG

all_unique_vectors = torch.vstack(train_dataset).unique(dim=0)
learned_unique_vectors = model.token_neural_map.unique(dim=0)
print(all_unique_vectors.shape, learned_unique_vectors.shape)
print()

all_unique_vectors = {tuple(row.round(decimals=2).cpu().numpy()) for row in all_unique_vectors}
learned_unique_vectors = {
    tuple(row.round(decimals=2).cpu().numpy()) for row in learned_unique_vectors
}
print(len(all_unique_vectors), len(learned_unique_vectors))
print()

inter = all_unique_vectors.intersection(learned_unique_vectors)
diff = all_unique_vectors - learned_unique_vectors
print(len(inter), len(diff))
print()